source: https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb

# Save and restore models  
(Tony) Keras 儲存和載入模型  

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.  
(Tony) 訓練過程中可以進行模型的儲存, 針對長時間的訓練就可以繼續之前完成的進度  
(Tony) 你也可以分享訓練完成的模型和復現之前完成的成果  

### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving).

## Setup

### Installs and imports  
(Tony) 環境設定  

Install and import TensorFlow and dependencies:  
(Tony) 注意你的 notebook 選用的 kernel 版本 (python2 or python3)  
(Tony) 如果 python3 請修改下面 code (pip -> pip3)  

In [1]:
!pip install h5py pyyaml 

The directory '/home/A50130/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/A50130/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


### Get an example dataset

We'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:  
(Tony) 我們來用 MNIST 資料集訓練模型並示範如何把權重儲存起來  
(Tony) 為加速示範速度, 只提取資料集中的前 1000 data  

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.4.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print("\ntrain_images.shape:", (train_images.shape))

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
print("train_images.shape:", (train_images.shape))


train_images.shape: (60000, 28, 28)
train_images.shape: (1000, 784)


### Define a model

Let's build a simple model we'll use to demonstrate saving and loading weights.  
(Tony) 建立一個簡單模型用來示範如何儲存和載入權重  

In [3]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Save checkpoints during training

The primary use case is to automatically save checkpoints *during* and at *the end* of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

`tf.keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.  
(Tony) 訓練結束是個重要的檢查點, 建立訓練過程中的檢查點, 可以在訓練意外被中斷後, 從最後的檢查點繼續訓練  
(Tony) `tf.keras.callbacks.ModelCheckpoint` 這個函數可以幫你建立檢查點  

### Checkpoint callback usage

Train the model and pass it the `ModelCheckpoint` callback:  
(Tony) 注意: 下面 code "training_1/" 這個 folder 要手動先建立, 不然會出現執行錯誤的問題  

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s - loss: 1.1730 - acc: 0.6610 - val_loss: 0.7235 - val_acc: 0.7790
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.4236 - acc: 0.8820 - val_loss: 0.5630 - val_acc: 0.8280
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.2869 - acc: 0.9280 - val_loss: 0.5029 - val_acc: 0.8420
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.2176 - acc: 0.9560 - val_loss: 0.4336 - val_acc: 0.8640
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.1547 - acc: 0.9650 - val_loss: 0.4231 - val_acc: 0.8620
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.1013 - acc: 0.9870 - val_loss: 0.4282 - val_acc: 0.8630
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.0827 - acc: 0.9890 - val_loss: 0.4183 - val_acc: 0.8560
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.0628 - a

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:  
(Tony) 上面 code 檢查點 callback function 會在每回合(epoch)完成後進行權重儲存  
(Tony) 下面檢查

In [5]:
!ls -la {checkpoint_dir}

total 1604
drwxr-xr-x 2 root root      29 Sep  6 15:53 .
drwxr-xr-x 4 root root     315 Sep  7 23:12 ..
-rw-r--r-- 1 root root 1641848 Sep  7 23:13 cp.ckpt


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):  
(Tony) 重新建一個未訓練的模型  
(Tony) 注意載入權重檔案必須是完全相同的模型才能共用  
(Tony) 用測試資料在這新的模型上驗證看看, 準確率非常地糟 (~10%)  

In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("\nUntrained model, accuracy: {:5.2f}%".format(100*acc))

  32/1000 [..............................] - ETA: 1s
Untrained model, accuracy: 11.20%


Then load the weights from the checkpoint, and re-evaluate:  
(Tony) 這次從檢查點載入權重, 並重新驗證看看  
(Tony) 立刻恢復到訓練後的水準(> 85%)  

In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("\nRestored model, accuracy: {:5.2f}%".format(100*acc))

  32/1000 [..............................] - ETA: 0s
Restored model, accuracy: 87.00%


### Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:  
(Tony) callback 提供一些有用的選項  
(Tony) 示範訓練一個新模型, 每 5 epochs 設為檢查點, 並且取個新檔名儲存權重  
(Tony) 注意: "training_2/" folder 必須事先建立好  

In [8]:
!mkdir training_2

In [9]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

Epoch 00004: saving model to training_2/cp-0004.ckpt
Epoch 00009: saving model to training_2/cp-0009.ckpt
Epoch 00014: saving model to training_2/cp-0014.ckpt
Epoch 00019: saving model to training_2/cp-0019.ckpt
Epoch 00024: saving model to training_2/cp-0024.ckpt
Epoch 00029: saving model to training_2/cp-0029.ckpt
Epoch 00034: saving model to training_2/cp-0034.ckpt
Epoch 00039: saving model to training_2/cp-0039.ckpt
Epoch 00044: saving model to training_2/cp-0044.ckpt
Epoch 00049: saving model to training_2/cp-0049.ckpt


Now, have a look at the resulting checkpoints (sorting by modification date):

In [14]:
!ls -la training_2/

total 16044
drwxr-xr-x 2 root root     250 Sep  7 23:14 .
drwxr-xr-x 5 root root    4096 Sep  7 23:16 ..
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0004.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0009.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0014.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0019.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0024.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0029.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0034.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0039.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0044.ckpt
-rw-r--r-- 1 root root 1641848 Sep  7 23:14 cp-0049.ckpt


In [11]:
!pip install pathlib

The directory '/home/A50130/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/A50130/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |################################| 51kB 865kB/s ta 0:00:01
  Running setup.py install for pathlib ... done


In [27]:
import pathlib

# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.ckpt")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints
print(latest)

training_2/cp-0049.ckpt


Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:  
(Tony) 重置模型並且載入最後一個檢查點(cp-0049.ckpt)測試看看  

In [29]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("\nRestored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s     

Restored model, accuracy: 87.90%


## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

* One or more shards that contain your model's weights. 
* An index file that indicates which weights are stored in which shard.  

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`  
(Tony) 可能 DNN farm 安裝的 tensorflow 太舊(1.4.0), 並沒有找到 shard file  

## Manually save weights

Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the `Model.save_weights` method.  
(Tony) 除了 callback 外, 也可以用 `Model.save_weights` 很簡單的儲存權重  

In [31]:
!mkdir checkpoints

In [33]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("\nRestored model, accuracy: {:5.2f}%".format(100*acc))

  32/1000 [..............................] - ETA: 0s
Restored model, accuracy: 87.90%


## Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.  
(Tony) 也可以把整個模型都儲存起來, 還包含了權重, 模型配置, 以及最佳化物件的配置  
(Tony) 如此可以讓我們繼續上次未完成的訓練動作  

Saving a fully-functional model in Keras is very useful—you can load them in [TensorFlow.js](https://js.tensorflow.org/tutorials/import-keras.html) and then train and run them in web browsers.  
(Tony) 在 keras 上儲存完整模型有個非常有用的地方, 你可以用 `TensorFlow.js` 直接在瀏覽器中載入模型使用  

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob.  
(Tony) Keras 採用 HDF5 標準儲存模型, 副檔名(.h5)  

In [34]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s - loss: 1.1858 - acc: 0.6580     
Epoch 2/5
1000/1000 [==============================] - 0s - loss: 0.4428 - acc: 0.8710     
Epoch 3/5
1000/1000 [==============================] - 0s - loss: 0.2917 - acc: 0.9250     
Epoch 4/5
1000/1000 [==============================] - 0s - loss: 0.2080 - acc: 0.9470     
Epoch 5/5
1000/1000 [==============================] - 0s - loss: 0.1573 - acc: 0.9650     


Now recreate the model from that file:  
(Tony) 從 h5 file 中載入並重建模型, 裡面已經包含權重了  

In [35]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:  
(Tony) 確認看看準確率  

In [37]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("\nRestored model, accuracy: {:5.2f}%".format(100*acc))

  32/1000 [..............................] - ETA: 0s
Restored model, accuracy: 86.40%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.  
(Tony) 目前 Keras 並不支援儲存 tensorflow 的最佳化物件  
(Tony) 當你載入並且要使用最佳化物件的話, 在載入後必須重新編譯模型, 而且你會遺失所有最佳化物件的狀態  